Create a notebook or python script named advanced_dataframes to do your work in for these exercises. [check]

Run python -m pip install pymysql from your terminal to install the mysql client (any folder is fine) [check]

cd into your exercises folder for this module and run echo env.py >> .gitignore [check]


### Create a function named get_db_url.   
    
It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

 #### Use your function to obtain a connection to the employees database.

In [23]:
from env import get_db_url

import pandas as pd
import numpy as np 


In [24]:
url = get_db_url("employees")


# f"
# format string (f-string) allows us to evaluate variables inside of curly braces
# real world application = makes writing strings with variables easier 



### Once you have successfully run a query:

a. Intentionally make a typo in the database url. What kind of error message do you see?

b. Intentionally make an error in your SQL query. What does the error message look like?



In [25]:
#a 
pd.read_sql('''SELECT * FROM employees LIMIT 5 OFFSET 50''', url)

query = '''
SELECT * FROM employees LIMIT 10
'''

pd.read_sql(query, url)

#b



,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
5,10006,1953-04-20,Anneke,Preusig,F,1989-06-02
6,10007,1957-05-23,Tzvetan,Zielinski,F,1989-02-10
7,10008,1958-02-19,Saniya,Kalloufi,M,1994-09-15
8,10009,1952-04-19,Sumant,Peac,F,1985-02-18
9,10010,1963-06-01,Duangkaew,Piveteau,F,1989-08-24


### Read the employees and titles tables into two separate DataFrames.

How many rows and columns do you have in each DataFrame? Is that what you expected?


In [31]:
distinctquery = '''
SELECT DISTINCT title FROM titles
'''

pd.read_sql(distinctquery, get_db_url("employees"))
                                                        
                                                        

,title
0,Senior Engineer
1,Staff
2,Engineer
3,Senior Staff
4,Assistant Engineer
5,Technique Leader
6,Manager


### Display the summary statistics for each DataFrame.

How many unique titles are in the titles DataFrame?

In [22]:
pd.read_sql('''SELECT DISTINCT titles FROM titles''', get_db_url("employees"))

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'titles' in 'field list'")
[SQL: SELECT DISTINCT titles FROM titles]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

#### c. What is the oldest date in the to_date column?

#### d. What is the most recent date in the to_date column?